# Tarea de programación: Base de datos de redes sociales

## ¡Bienvenido!

En este módulo has estado aprendiendo cómo los LLM pueden ayudar con muchas tareas comunes al trabajar con bases de datos. En esta tarea tendrás la oportunidad de poner en práctica esas habilidades. Estarás trabajando con una base de datos diseñada para representar una red social simple. Aquí están las tareas que necesitarás completar:

1. Escribir código que lea datos de un archivo CSV proporcionado en la base de datos
2. Desarrollar funciones que consulten la base de datos

### Envío y calificación

Este cuaderno será calificado en base a cuatro funciones que escribas. Puedes encontrar código inicial para las cuatro funciones a continuación. Se proporcionan pruebas unitarias para ayudarte a probar tu trabajo.

### Trabajando con el LLM:

- **GPT-4o está disponible:** Puedes usar el LLM que prefieras en esta tarea, pero [GPT-4o ha sido puesto a disposición](https://www.coursera.org/learn/ai-powered-software-and-system-design/ungradedLab/rSPHu/gpt-4o-environment-for-assignment-to-use-alongside-the-programming-assignment) en el laboratorio no calificado que sigue a esta tarea.
- **Proporciona contexto a tu LLM:** El LLM necesitará el contexto del problema en el que estás trabajando y el código ya disponible para ti para ayudar a desarrollar soluciones.
- **Trabaja de forma iterativa y prueba a medida que avanzas:** Recuerda, los consejos del LLM no siempre serán perfectos. Depende de ti decidir qué es útil, probar el código que proporciona e iterar a medida que trabajas hacia una solución.


### Importaciones necesarias


In [1]:
import unittests
import submission_checker

### Esquema de la base de datos

La función a continuación define el esquema para la base de datos que utilizarás en esta tarea. La base de datos representa una red social compuesta por personas que pueden ser amigas entre sí y unirse a clubes. No puedes editar esta función, pero debes entender cómo funciona y el esquema de la base de datos que define. **Lee esta función cuidadosamente y compártela con tu LLM** para asegurarte de que entiendes el esquema antes de continuar.


In [2]:
from sqlalchemy import create_engine, Column, Integer, String, ForeignKey, Table
from sqlalchemy.orm import declarative_base
from sqlalchemy.orm import relationship, sessionmaker
import numpy as np
import os
import pandas as pd
import json

# Creates the database for the social network

def create_database():
    Base = declarative_base()

    friendships = Table('friendships', Base.metadata,
                        Column('person_id', Integer, ForeignKey('people.id'), primary_key=True),
                        Column('friend_id', Integer, ForeignKey('people.id'), primary_key=True))

    club_members = Table('club_members', Base.metadata,
                         Column('person_id', Integer, ForeignKey('people.id'), primary_key=True),
                         Column('club_id', Integer, ForeignKey('clubs.id'), primary_key=True))

    class Person(Base):
        __tablename__ = 'people'
        id = Column(Integer, primary_key=True)
        name = Column(String)
        age = Column(Integer)
        gender = Column(String)
        location = Column(String)
        friends = relationship("Person",
                               secondary=friendships,
                               primaryjoin=id == friendships.c.person_id,
                               secondaryjoin=id == friendships.c.friend_id)
        clubs = relationship("Club", secondary=club_members, back_populates="members")

    class Club(Base):
        __tablename__ = 'clubs'
        id = Column(Integer, primary_key=True)
        description = Column(String)
        members = relationship("Person", secondary=club_members, back_populates="clubs")

    if os.path.exists("social_network.db"):
        os.remove("social_network.db")
    engine = create_engine(f'sqlite:///{"social_network.db"}', echo=False)
    Base.metadata.create_all(engine)

    Session = sessionmaker(bind=engine)
    session = Session()

    return session, Club, Person, friendships, club_members

### Ejercicio 1: Cargar la base de datos desde un archivo CSV

La base de datos definida en la función `create_database` aún no contiene ningún dato. Necesitarás escribir el código para la función `load_data_from_csv` a continuación para poblar la base de datos. Para hacer eso, utilizarás el archivo `members.csv` que está disponible en este laboratorio en el mismo directorio que este cuaderno. Los datos están almacenados en una tabla única con una fila para cada persona. La columna Friendships contiene los IDs de todas las personas que la persona en esa fila considera amigos. La columna Clubs contiene los nombres de cada club al que pertenece la persona en esa fila.

Algunas notas importantes sobre este ejercicio:

* Para que tu función sea calificada correctamente, no cambies el nombre de la función ni sus parámetros.
* No deberías necesitar importar bibliotecas adicionales, ya que a veces esto puede causar errores con el calificador. Si es absolutamente necesario, hazlo dentro de la misma celda que la función `load_data_from_csv`.

**SUGERENCIAS**:

- **Recuerda** que debes evitar pegar código de LLM directamente en tu solución. Si un código no se ejecuta, intenta mirar la excepción y entender qué podría estar sucediendo, luego puedes pasar más información al LLM.
- **Recuerda** también pasar la función `create_database()` para que el LLM pueda entender la estructura de la base de datos.
- **Recuerda** pasar la información necesaria sobre la estructura de `members.csv`. Puedes verla haciendo clic en el archivo `members.csv` en el panel derecho. **Pasa la información correcta al LLM sobre las columnas,** **recuerda mencionar que la columna Friends es una lista de enteros correspondientes al ID de los amigos y la columna Clubs es una lista de cadenas correspondientes a los clubes a los que pertenece el miembro.**
- **Pasa** la función `load_data_from_csv` y pídele que la complete.
- **Recuerda** que la solución debe colocarse en la celda designada para ello (la que está debajo de aquí). Puedes crear nuevas celdas para probar si lo deseas, pero **recuerda** pegar tu solución en la celda inicial calificada. **NO** la elimines, ya que está etiquetada y el autocalificador buscará la etiqueta específica, eliminarla hará que el autocalificador no pueda encontrar tu solución.


In [3]:
# The members.csv file contains 20 people. The code below will show you the first 5 rows to help you understand the format of the CSV file

pd.read_csv("members.csv", converters = {'Friendships': eval, "Clubs": eval}).head()

,ID,Name,Surname,Location,Age,Gender,Friendships,Clubs
0,0,John,Rocha,"948 Christian Park Apt. 683 New Christopher, M...",57,Male,"[4, 10, 12, 5, 19, 14]",[Fitness Club]
1,1,William,Ruiz,"09477 Paul Station Williamsmouth, NM 93899",42,Female,"[9, 15, 17, 10, 3]","[Travel Club, Art Club, Cooking Club]"
2,2,Jackie,Mccullough,"903 Keller River Suite 149 Davidside, IN 11476",36,Non-binary,"[6, 4, 10, 9]","[Hiking Club, Art Club]"
3,3,Michael,Powell,"7528 Brenda Mills Jenniferstad, RI 38209",67,Female,"[11, 7, 13, 6, 14, 2]","[Hiking Club, Chess Club, Gaming Club, Book Cl..."
4,4,Scott,Boyd,"545 Evelyn Shores Apt. 744 North Craigchester,...",47,Female,"[15, 17, 1, 11, 0, 10, 5]",[Gaming Club]


In [ ]:
# GRADED CELL 1 - Do NOT delete it, do NOT place your solution anywhere else. You can create new cells and work from there, but in the end add your solution in this cell.
# Function to load data from CSV into the database
def load_data_from_csv(session, Club, Person, friendships, club_members, csv_path="members.csv"):
    """
    Load data from a CSV file into the database, clearing existing data and creating new records for people, clubs, friendships, and club memberships.

    This function performs several key operations:
    1. Clears existing data from the Person, Club, friendships, and club_members tables.
    2. Reads data from a CSV file specified by `csv_path`, defaulting to "members.csv".
    3. For each row in the CSV, it creates or retrieves clubs mentioned, creates a person with the specified attributes, and associates them with the clubs.
    4. Establishes friendships based on the "Friendships" column in the CSV, which lists friend IDs for each person.
    5. Commits all changes to the database to ensure data integrity and consistency.

    Parameters:
    - session: The SQLAlchemy session object used for database transactions.
    - Club: The Club class model used to create or retrieve club records.
    - Person: The Person class model used to create person records.
    - friendships: The table/model representing friendships between people.
    - club_members: The table/model representing memberships of people in clubs.
    - csv_path (str, optional): The path to the CSV file containing the data to be loaded. Defaults to "members.csv".

    Returns:
    None. The function operates by side effects, modifying the database directly.

    Note:
    The function assumes the CSV file is formatted with specific columns: "ID", "Name", "Surname", "Age", "Gender", "Location", "Clubs", and "Friendships".
    "Clubs" is expected to be string representations of lists and "Friendships" is expected to be a list of IDs representing the member friends.
    """
    # Step 1: Clear existing data from all relevant tables
    session.query(Person).delete()
    session.query(Club).delete()
    session.query(friendships).delete()
    session.query(club_members).delete()

    session.commit()  # Commit the deletion of all existing records

    # Load the CSV data
    df = pd.read_csv("members.csv", converters = {'Friendships': eval, "Clubs": eval})
    ### START CODE HERE ###
    pass

In [4]:
# GRADED CELL 1 - Do NOT delete it, do NOT place your solution anywhere else. You can create new cells and work from there, but in the end add your solution in this cell.
# Function to load data from CSV into the database
def load_data_from_csv(session, Club, Person, friendships, club_members, csv_path="members.csv"):
    """
    Load data from a CSV file into the database, clearing existing data and creating new records for people, clubs, friendships, and club memberships.
    """

    # Step 1: Clear existing data from all relevant tables
    session.query(Person).delete()
    session.query(Club).delete()
    session.query(friendships).delete()
    session.query(club_members).delete()

    session.commit()  # Commit the deletion of all existing records

    # Load the CSV data
    df = pd.read_csv(csv_path, converters={'Friendships': eval, "Clubs": eval})

    # Diccionarios auxiliares para evitar duplicados
    clubs_dict = {}
    people_dict = {}

    # Crear objetos Person y Club
    for _, row in df.iterrows():
        # Crear o reutilizar clubs
        person_clubs = []
        for club_desc in row["Clubs"]:
            if club_desc not in clubs_dict:
                new_club = Club(description=club_desc)
                session.add(new_club)
                session.flush()  # Para que se le asigne un ID
                clubs_dict[club_desc] = new_club
            person_clubs.append(clubs_dict[club_desc])

        # Crear persona
        person = Person(
            id=row["ID"],
            name=f"{row['Name']} {row['Surname']}",
            age=row["Age"],
            gender=row["Gender"],
            location=row["Location"],
            clubs=person_clubs
        )
        session.add(person)
        people_dict[row["ID"]] = person

    session.commit()  # Guardamos personas y clubes

    # Segunda pasada para las amistades (relaciones recursivas)
    for _, row in df.iterrows():
        person = people_dict[row["ID"]]
        for friend_id in row["Friendships"]:
            if friend_id in people_dict:
                friend = people_dict[friend_id]
                if friend not in person.friends:  # Evita duplicados
                    person.friends.append(friend)

    session.commit()


In [5]:
# The code below creates the database and reads in the data 
session, Club, Person, friendships, club_members = create_database()
load_data_from_csv(session, Club, Person, friendships, club_members, "members.csv")

# If your load_data_from_csv function is working correctly, then you should have read in data correctly into all four tables in the database.

print_amount = 3

# Print first 3 persons
print("=== All Persons ===")
people = session.query(Person).all()
for person in people[:print_amount ]:
    print(f"ID: {person.id}, Name: {person.name}, Age: {person.age}, Gender: {person.gender}, Location: {person.location}")

# Print first 3 clubs and their members
print("\n=== All Clubs and their Members ===")
clubs = session.query(Club).all()
for club in clubs[:print_amount ]:
    print(f"Club ID: {club.id}, Description: {club.description}, Members: {[member.name for member in club.members]}")

# Print friendships of first three persons
print("\n=== Friendships ===")
for person in people[:print_amount ]:
    friends = [friend.name for friend in person.friends]
    print(f"{person.name}'s Friends: {friends}")

=== All Persons ===
ID: 0, Name: John Rocha, Age: 57, Gender: Male, Location: 948 Christian Park Apt. 683 New Christopher, MN 06675
ID: 1, Name: William Ruiz, Age: 42, Gender: Female, Location: 09477 Paul Station Williamsmouth, NM 93899
ID: 2, Name: Jackie Mccullough, Age: 36, Gender: Non-binary, Location: 903 Keller River Suite 149 Davidside, IN 11476

=== All Clubs and their Members ===
Club ID: 1, Description: Fitness Club, Members: ['John Rocha', 'Amanda Norris', 'Michael Clark', 'Christina Murphy']
Club ID: 2, Description: Travel Club, Members: ['William Ruiz', 'Michael Powell', 'Becky Peterson', 'Nicholas Harrington', 'Luis Kim', 'Nathan Mendez']
Club ID: 3, Description: Art Club, Members: ['William Ruiz', 'Jackie Mccullough', 'Amanda Norris', 'Becky Peterson', 'Mark Allen', 'Brian Mays', 'Eric Dougherty', 'Nathan Mendez']

=== Friendships ===
John Rocha's Friends: ['Scott Boyd', 'Andrew Williams', 'Nicholas Harrington', 'Christina Murphy', 'Luis Kim', 'Mark Mcintyre']
William Ru

Resultado esperado:
```
=== Todas las personas ===
ID: 0, Nombre: John Rocha, Edad: 57, Género: Masculino, Ubicación: 948 Christian Park Apt. 683 New Christopher, MN 06675
ID: 1, Nombre: William Ruiz, Edad: 42, Género: Femenino, Ubicación: 09477 Paul Station Williamsmouth, NM 93899
ID: 2, Nombre: Jackie Mccullough, Edad: 36, Género: No binario, Ubicación: 903 Keller River Suite 149 Davidside, IN 11476

=== Todos los clubes y sus miembros ===
ID del Club: 1, Descripción: Club de Fitness, Miembros: ['John Rocha', 'Amanda Norris', 'Michael Clark', 'Christina Murphy']
ID del Club: 2, Descripción: Club de Viajes, Miembros: ['William Ruiz', 'Michael Powell', 'Becky Peterson', 'Nicholas Harrington', 'Luis Kim', 'Nathan Mendez']
ID del Club: 3, Descripción: Club de Arte, Miembros: ['William Ruiz', 'Jackie Mccullough', 'Amanda Norris', 'Becky Peterson', 'Mark Allen', 'Brian Mays', 'Eric Dougherty', 'Nathan Mendez']

=== Amistades ===
Amigos de John Rocha: ['Scott Boyd', 'Andrew Williams', 'Nicholas Harrington', 'Christina Murphy', 'Luis Kim', 'Mark Mcintyre']
Amigos de William Ruiz: ['Michael Powell', 'Mark Allen', 'Nicholas Harrington', 'Brian Mays', 'Nathan Mendez']
Amigos de Jackie Mccullough: ['Scott Boyd', 'Amanda Norris', 'Mark Allen', 'Nicholas Harrington']
```


In [6]:
unittests.test_load_data_from_csv(load_data_from_csv)

 All tests passed!
  If you made your solution in a different cell, make sure to include it in the graded cell as well.


### Ejercicio 2: Obtener los miembros de un club

Suponiendo que tus datos se han cargado correctamente, ahora deberías poder consultar los datos en tu base de datos. Para este ejercicio, escribe una función llamada `get_club_members`. Esta función debe aceptar una descripción de un club y una sesión, y devolver una lista de todos sus miembros. **Asegúrate de que esta función devuelva una lista que contenga los objetos Person definidos.** Solo debe ingresar una **descripción del club**.


In [ ]:
# GRADED CELL 2 - Do NOT delete it, do NOT place your solution anywhere else. You can create new cells and work from there, but in the end add your solution in this cell.
def get_club_members(session, club_description):
    """
    Returns a list of Person objects who are members of a club given the club's description.
    
    Parameters:
    - session: The SQLAlchemy session for database queries.
    - club_description (str): The description of the club for which members are to be retrieved.
    
    Returns:
    - List[Person]: A list of Person objects who are members of the specified club.
    """
    ### START CODE HERE ###
    pass

In [7]:
def get_club_members(session, club_description):
    """
    Returns a list of Person objects who are members of a club given the club's description.
    """
    club = session.query(Club).filter_by(description=club_description).first()
    if club:
        return club.members
    else:
        return []


In [8]:
# Example usage of the get_club_members function

# Assume the session and all models have been correctly set up and populated as per your initial code

# Fetching members of the "Hiking Club"
hiking_club_members = get_club_members(session, "Hiking Club")

# Printing out the names of all members of the Hiking Club
print("Members of the Hiking Club:")
for person in hiking_club_members:
    print(f"- {person.name}, Age: {person.age}, Location: {person.location}")

Members of the Hiking Club:
- Jackie Mccullough, Age: 36, Location: 903 Keller River Suite 149 Davidside, IN 11476
- Michael Powell, Age: 67, Location: 7528 Brenda Mills Jenniferstad, RI 38209
- Amanda Norris, Age: 27, Location: 8446 Thomas Crossing Suite 184 West Jacobshire, CA 41663
- Michael Clark, Age: 39, Location: 688 Sean River Apt. 585 Amandatown, AZ 76721
- Christina Murphy, Age: 19, Location: 276 Wolfe Springs Lake Eric, PA 61512
- Luis Kim, Age: 34, Location: 23367 Chase Heights Apt. 892 Calebchester, NC 59073
- Nathan Mendez, Age: 70, Location: 16039 Carey Views Apt. 776 Tylerfurt, DE 67034


Miembros del Club de Excursionismo:
- Jackie Mccullough, Edad: 36, Ubicación: 903 Keller River Suite 149 Davidside, IN 11476
- Michael Powell, Edad: 67, Ubicación: 7528 Brenda Mills Jenniferstad, RI 38209
- Amanda Norris, Edad: 27, Ubicación: 8446 Thomas Crossing Suite 184 West Jacobshire, CA 41663
- Michael Clark, Edad: 39, Ubicación: 688 Sean River Apt. 585 Amandatown, AZ 76721
- Christina Murphy, Edad: 19, Ubicación: 276 Wolfe Springs Lake Eric, PA 61512
- Luis Kim, Edad: 34, Ubicación: 23367 Chase Heights Apt. 892 Calebchester, NC 59073
- Nathan Mendez, Edad: 70, Ubicación: 16039 Carey Views Apt. 776 Tylerfurt, DE 67034


In [9]:
unittests.test_get_club_members(load_data_from_csv, get_club_members)

 All tests passed!
  If you made your solution in a different cell, make sure to include it in the graded cell as well.


### Ejercicio 3: Obtener los amigos de una persona

En este ejercicio, se requiere crear una función llamada `get_friends_of_person`. Esta función debe aceptar el nombre de una persona y una sesión, devolver una lista de todas las personas que consideran ser amigos. **Asegúrate de que esta función devuelva una lista que contenga los objetos Person definidos.** La entrada debe ser solo el **nombre de una persona**.


In [ ]:
# GRADED CELL 3 - Do NOT delete it, do NOT place your solution anywhere else. You can create new cells and work from there, but in the end add your solution in this cell.
def get_friends_of_person(session, person_name):
    """
    Returns a list of Person objects who are friends with the specified person.
    
    Parameters:
    - session: The SQLAlchemy session object used to query the database.
    - person_name (str): The name of the person for whom to retrieve friends.
    
    Returns:
    - List[Person]: A list of Person objects who are friends with the specified person.
    """
    ### START CODE HERE ###
    pass

In [10]:
# GRADED CELL 3 - Do NOT delete it, do NOT place your solution anywhere else. You can create new cells and work from there, but in the end add your solution in this cell.
def get_friends_of_person(session, person_name):
    """
    Returns a list of Person objects who are friends with the specified person.
    """
    person = session.query(Person).filter_by(name=person_name).first()
    if person:
        return person.friends
    else:
        return []


In [11]:
# Example usage of the get_friends_of_person function

# Fetching friends of given name
name = "John Rocha"

john_friends = get_friends_of_person(session,name)

# Printing out the names of all friends of John Rocha
print(f"Friends of {name}:")
for friend in john_friends:
    print(f"- {friend.name}, Age: {friend.age}, Location: {friend.location}")

Friends of John Rocha:
- Scott Boyd, Age: 47, Location: 545 Evelyn Shores Apt. 744 North Craigchester, CO 47369
- Andrew Williams, Age: 24, Location: USCGC Jacobs FPO AE 78539
- Nicholas Harrington, Age: 52, Location: 97681 Hernandez Villages Suite 344 Elizabethborough, UT 44717
- Christina Murphy, Age: 19, Location: 276 Wolfe Springs Lake Eric, PA 61512
- Luis Kim, Age: 34, Location: 23367 Chase Heights Apt. 892 Calebchester, NC 59073
- Mark Mcintyre, Age: 22, Location: 5635 Jennifer Freeway Suite 436 Newmanberg, IL 05315


Resultado esperado:
```
- Scott Boyd, Edad: 47, Ubicación: 545 Evelyn Shores Apt. 744 North Craigchester, CO 47369
- Andrew Williams, Edad: 24, Ubicación: USCGC Jacobs FPO AE 78539
- Nicholas Harrington, Edad: 52, Ubicación: 97681 Hernandez Villages Suite 344 Elizabethborough, UT 44717
- Christina Murphy, Edad: 19, Ubicación: 276 Wolfe Springs Lake Eric, PA 61512
- Luis Kim, Edad: 34, Ubicación: 23367 Chase Heights Apt. 892 Calebchester, NC 59073
- Mark Mcintyre, Edad: 22, Ubicación: 5635 Jennifer Freeway Suite 436 Newmanberg, IL 05315
```


In [12]:
unittests.test_get_friends_of_person(load_data_from_csv, get_friends_of_person)

 All tests passed!
  If you made your solution in a different cell, make sure to include it in the graded cell as well.


### Ejercicio 4: Obtener a todas las personas que consideran a una persona como amiga

Ahora escribe una función llamada `get_persons_who_consider_them_friend`. Esta función debe tomar dos parámetros: el nombre de un individuo y una sesión. Devolverá una lista de personas que consideran a este individuo como amigo. Es importante recordar que en nuestra base de datos, la amistad no es necesariamente mutua. Por ejemplo, Alice podría considerar a Bob como amigo, pero Bob podría no sentir lo mismo por Alice. **Tu función debe devolver una lista de objetos Person para todos los que consideran al nombre de entrada como su amigo.** La entrada a esta función debe ser estrictamente el **nombre de la persona** sobre la que estás preguntando.


In [ ]:
# GRADED CELL 4 - Do NOT delete it, do NOT place your solution anywhere else. You can create new cells and work from there, but in the end add your solution in this cell.
def get_persons_who_consider_them_friend(session, person_name):
    """
    Returns a list of Person objects who consider the specified person as their friend,
    in a scenario where friendships are unidirectional.
    
    Parameters:
    - person_name (str): The name of the person to find who is considered as a friend by others.
    
    Returns:
    - List[Person]: A list of Person objects who consider the specified person as their friend.
    """
    ### START CODE HERE ###
    pass

In [13]:
# GRADED CELL 4 - Do NOT delete it, do NOT place your solution anywhere else. You can create new cells and work from there, but in the end add your solution in this cell.
def get_persons_who_consider_them_friend(session, person_name):
    """
    Returns a list of Person objects who consider the specified person as their friend,
    in a scenario where friendships are unidirectional.
    """
    target = session.query(Person).filter_by(name=person_name).first()
    if not target:
        return []
    
    people = session.query(Person).all()
    result = [p for p in people if target in p.friends]
    return result


In [14]:
# Example usage of the get_persons_who_consider_them_friend function

# Fetching people who consider given name as their friend
name = 'John Rocha'

name_friend_of = get_persons_who_consider_them_friend(session, name)

# Printing out the names of all people who consider Alice as their friend
print(f"People who consider {name} as their friend:")
for person in name_friend_of:
    print(f"- {person.name}, Age: {person.age}, Location: {person.location}")

People who consider John Rocha as their friend:
- Scott Boyd, Age: 47, Location: 545 Evelyn Shores Apt. 744 North Craigchester, CO 47369
- Christina Murphy, Age: 19, Location: 276 Wolfe Springs Lake Eric, PA 61512
- Brian Mays, Age: 61, Location: 80208 Parker Glen Harrisland, PW 54882
- Nathan Mendez, Age: 70, Location: 16039 Carey Views Apt. 776 Tylerfurt, DE 67034


Resultado esperado:
```
Personas que consideran a John Rocha como su amigo:
- Scott Boyd, Edad: 47, Ubicación: 545 Evelyn Shores Apt. 744 North Craigchester, CO 47369
- Christina Murphy, Edad: 19, Ubicación: 276 Wolfe Springs Lake Eric, PA 61512
- Brian Mays, Edad: 61, Ubicación: 80208 Parker Glen Harrisland, PW 54882
- Nathan Mendez, Edad: 70, Ubicación: 16039 Carey Views Apt. 776 Tylerfurt, DE 67034
```


In [15]:
unittests.test_get_persons_who_consider_them_friend(load_data_from_csv, get_persons_who_consider_them_friend)

 All tests passed!
  If you made your solution in a different cell, make sure to include it in the graded cell as well.


## Preparando tu Envío para la Calificación

Tu envío será evaluado por un sistema de calificación automatizado, conocido como un autograder. Este sistema revisa automáticamente tu cuaderno y asigna una calificación basada en criterios específicos. Es importante tener en cuenta que el autograder solo evaluará las celdas marcadas para la calificación y no considerará el contenido de todo el cuaderno. Por lo tanto, si incluyes algún contenido adicional (como declaraciones de impresión) fuera de las funciones en las celdas calificadas, podría interrumpir el proceso del autograder. Esta discrepancia podría ser la razón por la cual podrías pasar todas las pruebas unitarias pero aún encontrar problemas con el autograder.

Para evitar tales problemas, por favor ejecuta la siguiente celda antes de enviar. Este paso verificará la consistencia dentro de las celdas calificadas pero no evaluará la corrección de tus soluciones—ese aspecto es determinado por las pruebas unitarias. Si la verificación de consistencia descubre algún problema, tendrás la oportunidad de revisar y ajustar tu código en consecuencia.

**Recuerda, esta verificación se enfoca en asegurar que las celdas calificadas estén correctamente formateadas y no evalúa la precisión de tus respuestas.**


In [ ]:
submission_checker.check_notebook()

Una vez que hayas escrito los cuatro métodos, estás listo para enviar. **Asegúrate de guardar tu trabajo antes de enviarlo** para que el evaluador pueda evaluar la versión más reciente de tu tarea.

¡Felicidades por terminar esta tarea!
